In [1]:
# import the needed libraries
import pandas as pd
import geopandas as gpd
import pyodbc
from shapely.geometry import Polygon, MultiPolygon
from shapely.geometry import mapping
import json
from geoalchemy2 import Geometry, WKTElement
import sqlalchemy as sal
from tqdm import tqdm
# from multiprocessing import Pool, cpu_count

C:\Users\Eslam\AppData\Local\Temp\ipykernel_19084\312472351.py:3: DeprecationWarning: Shapely 2.0 is installed, but because PyGEOS is also installed, GeoPandas still uses PyGEOS by default. However, starting with version 0.14, the default will switch to Shapely. To force to use Shapely 2.0 now, you can either uninstall PyGEOS or set the environment variable USE_PYGEOS=0. You can do this before starting the Python process, or in your code before importing geopandas:

import os
os.environ['USE_PYGEOS'] = '0'
import geopandas

In the next release, GeoPandas will switch to using Shapely by default, even if PyGEOS is installed. If you only have PyGEOS installed to get speed-ups, this switch should be smooth. However, if you are using PyGEOS directly (calling PyGEOS functions on geometries from GeoPandas), this will then stop working and you are encouraged to migrate from PyGEOS to Shapely 2.0 (https://shapely.readthedocs.io/en/latest/migration_pygeos.html).
  import geopandas as gpd


In [2]:
# reading the table that contains the attributes of the parcels, it is has been simplified in ArcGIS Pro Pro before exported in a csv file
table = pd.read_csv('data_table1.csv')
table.shape[0]

1487115

In [3]:
# reading the first chunk that exported from the source dataset as a GeoDataFrame
# it is exported from ArcGIS Pro with removing the Z dimention from the geometry
#p1 = gpd.read_file('parcels.gdb', driver="OpenFileGDB", layer='chunk_1_100000_v2')
# p2 = gpd.read_file('parcels.gdb', driver="OpenFileGDB", layer='chunk_2_100000_v2')
# p3 = gpd.read_file('parcels.gdb', driver="OpenFileGDB", layer='chunk_3_50000')
# p4 = gpd.read_file('parcels.gdb', driver="OpenFileGDB", layer='chunk_4_50000')
# p5 = gpd.read_file('parcels.gdb', driver="OpenFileGDB", layer='chunk_5_100000')
# p6 = gpd.read_file('parcels.gdb', driver="OpenFileGDB", layer='chunk_6_100000')
# p8 = gpd.read_file('parcels.gdb', driver="OpenFileGDB", layer='chunk_7_200000')[0:100000]
#p8 = gpd.read_file('parcels.gdb', driver="OpenFileGDB", layer='chunk_7_200000')[100000:]
# p9 = gpd.read_file('parcels.gdb', driver="OpenFileGDB", layer='chunk_9_100000')
#p10 = gpd.read_file('parcels.gdb', driver="OpenFileGDB", layer='chunk_10_100000')
# p11 = gpd.read_file('parcels.gdb', driver="OpenFileGDB", layer='chunk_11_250000')
p12 = gpd.read_file('parcels.gdb', driver="OpenFileGDB", layer='chunk_12_246739')
p12.shape[0]

246739

In [4]:
# create a new column for SourceObjectID and fill it with sequenced numbers
#p1['SourceObjectID'] = range(100001, len(p1)+1)
# p2['SourceObjectID'] = range(100001, 100001 + 100000)
# p3['SourceObjectID'] = range(200001, 200001 + 50000)
# p4['SourceObjectID'] = range(250001, 250001 + 50000)
# p5['SourceObjectID'] = range(300001, 300001 + 100000)
#p6['SourceObjectID'] = range(400001, 400001 + 100000)
#p7['SourceObjectID'] = range(500001, 500001 + 100000)
# p8['SourceObjectID'] = range(600001, 600001 + 100000)
#p9['SourceObjectID'] = range(700001, 700001 + 100000)
#p10['SourceObjectID'] = range(800001, 800001 + 100000)
#p11['SourceObjectID'] = range(900001, 900001 + 250000)
p12['SourceObjectID'] = range(1150001, 1150001 + 246739)

In [5]:
# define the bad/empty geometries
empty_geometry_rows = p12[p12['geometry'].is_empty]

In [6]:
# print the count of the empty geometries
empty_geometry_rows.shape[0]

454

In [7]:
# empty_geometry_rows['SourceObjectID']

In [92]:
# export bad geometries in a file
# empty_geometry_rows.to_csv('Empty geometries.csv')

In [10]:
# # read the csv file of bad geometries
# bad_geoms = pd.read_csv('Empty geometries.csv')

In [11]:
# # append the empty geoms to the file
# bad_geoms = bad_geoms.append(empty_geometry_rows)
# bad_geoms.to_csv('Empty geometries.csv')

In [12]:
# drop the empty geometries from the current chunk/GeoDataFrame
p12 = p12.drop(empty_geometry_rows.index)
p12.shape[0]

246285

In [13]:
# join the csv table with the current parcels chunk/GeoDataFrame to get all of the needed columns
merged = pd.merge(p12, table, how='left', on=['CleanParcelID', 'FullLegalDescription', 'FullOwnerName', 'FullOwnerAddress', 'FullPhysicalAddress'])

In [14]:
merged.shape[0]

246285

In [15]:
merged.columns

Index(['CleanParcelID', 'Label', 'Suffix', 'CountyID', 'FullLegalDescription',
       'FullOwnerName', 'FullOwnerAddress', 'FullPhysicalAddress',
       'Shape_Length', 'Shape_Area', 'geometry', 'SourceObjectID', 'OID_',
       'ID', 'ParcelID', 'DistrictCode', 'TaxDistrict'],
      dtype='object')

In [16]:
# merged = merged.drop_duplicates(subset=['SourceObjectID'], keep='first')
# merged.shape[0]

In [18]:
merged.tail(2)

,CleanParcelID,Label,Suffix,CountyID,FullLegalDescription,FullOwnerName,FullOwnerAddress,FullPhysicalAddress,Shape_Length,Shape_Area,geometry,SourceObjectID,OID_,ID,ParcelID,DistrictCode,TaxDistrict
246283,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,754.052651,31096.678743,"MULTIPOLYGON (((614483.575 4334491.102, 614462...",1396738,NaN,NaN,NaN,NaN,NaN
246284,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,754.052651,31096.678743,"MULTIPOLYGON (((614483.575 4334491.102, 614462...",1396739,NaN,NaN,NaN,NaN,NaN


In [19]:
# drop some columns that are not needed to be loaded to the database
merged = merged.drop(columns=['CleanParcelID','OID_','Shape_Length','Shape_Area'])

# rename the remain columns to match the same names of the table in the database
merged.rename(columns={'ID':'SourceGISID', 'ParcelID':'SourceFullParcelID', 'DistrictCode':'DistrictID',
                      'TaxDistrict':'TaxMapID', 'Suffix':'SuffixID', 'FullOwnerName':'SourceFullOwnerName',
                      'FullOwnerAddress':'SourceFullOwnerAddress', 'FullPhysicalAddress':'SourceFullPhysicalAddress'}, inplace=True)

In [20]:
# assign values of SourceFullParcelID column to ParcelID column
merged['ParcelID'] = merged['SourceFullParcelID']
merged.rename(columns={'Label':'SourceLabelName'}, inplace=True)

In [21]:
# merged.tail()

In [22]:
# change the crs of the GeoDataFrame to EPSG:3857
merged = merged.to_crs('EPSG:3857')

In [23]:
# create a new empty string column [GeoJSON] to carry all of the values of all of the columns in a GeoJSON format later
merged['GeoJSON'] = ''

In [24]:
# assign the name of the source shapefile/dataset to the column DataSource to know the source of the data
merged['DataSource'] = 'WV_Parcels_All_Counties'

In [25]:
# create a 3 new string empty columns, these columns will be filled on the databse using get_date() SQL function
merged[['SourceDateTime','SysStartTime','SysEndTime']] = ''

In [26]:
# clean the NaN values by replacing them to 0
merged[['SourceGISID','TaxMapID','DistrictID']] = merged[['SourceGISID','TaxMapID','DistrictID']].fillna(0).astype(int)

In [27]:
# replace the 0 values with '' after converting the 3 columns into string
merged[['SourceGISID','TaxMapID','DistrictID']] = merged[['SourceGISID','TaxMapID','DistrictID']].astype(str)
merged[['SourceGISID','TaxMapID','DistrictID']] = merged[['SourceGISID','TaxMapID','DistrictID']].replace('0','')

In [28]:
# design a new columns order to make the GeoDataFrame matches the order of the table on the database
order = ['SourceObjectID','SourceGISID','SourceFullParcelID','CountyID','DistrictID','TaxMapID','ParcelID','SuffixID','FullLegalDescription',
         'SourceFullOwnerName','SourceFullOwnerAddress','SourceFullPhysicalAddress','SourceLabelName','GeoJSON','DataSource','SourceDateTime',
        'SysStartTime','SysEndTime','geometry']

# convert these columns into string to make them match the data types of the table on the database
merged[['SourceObjectID','SourceGISID','DistrictID','TaxMapID']] = merged[['SourceObjectID','SourceGISID','DistrictID','TaxMapID']].astype(str)

In [29]:
# assign the designed order to the GeoDataFrame
merged = merged[order]

In [30]:
# merged.head()

In [31]:
# clean the data by replace nan values with ''
merged = merged.replace('nan', '')
merged = merged.fillna('')
# merged.head()

In [32]:
# # fill the GeoJSON column
# for index, row in merged.iterrows():
#     properties = row.drop('geometry').to_dict()  # Exclude the geometry column
#     geometry = mapping(merged['geometry'].iloc[index])  # Convert MultiPolygon to GeoJSON-like dictionary
#     feature = {"type": "Feature","properties": properties,"geometry": geometry}
#     # Update the 'geojson' column with the GeoJSON string
#     merged.at[index, 'GeoJSON'] = json.dumps(feature)

In [33]:
for index, row in merged.iterrows():
    properties = row.drop('geometry').to_dict()  # Exclude the geometry column
    geometry = mapping(row['geometry'])  # Convert MultiPolygon to GeoJSON-like dictionary
    feature = {"type": "Feature","properties": properties,"geometry": geometry}
    # Update the 'geojson' column with the GeoJSON string
    merged.at[index, 'GeoJSON'] = json.dumps(feature)

In [34]:
# selected_row = merged.iloc[[87657]]
# selected_geodataframe = gpd.GeoDataFrame(selected_row, geometry='geometry')
# selected_geodataframe.to_file('bad value.geojson')

In [35]:
# create a new string geometry column and fill it with the values of the actual column of the GeoDataFrame
merged['geom'] = ''
for idx, row in merged.iterrows():
    merged.at[idx, 'geom'] = str(row['geometry'])

In [36]:
# checking for the large values of GeoJSON column
merged['length'] = merged['GeoJSON'].str.len()
top_10 = merged.nlargest(10, 'length')
top_10

,SourceObjectID,SourceGISID,SourceFullParcelID,CountyID,DistrictID,TaxMapID,ParcelID,SuffixID,FullLegalDescription,SourceFullOwnerName,...,SourceFullPhysicalAddress,SourceLabelName,GeoJSON,DataSource,SourceDateTime,SysStartTime,SysEndTime,geometry,geom,length
224940,1374954,,,10,,,,0002,,,...,,81E-9999.2,"{""type"": ""Feature"", ""properties"": {""SourceObje...",WV_Parcels_All_Counties,,,,"MULTIPOLYGON (((-9034091.680 4575137.027, -903...",MULTIPOLYGON (((-9034091.680348001 4575137.027...,10602464
239709,1389737,,,,,,,,,,...,,,"{""type"": ""Feature"", ""properties"": {""SourceObje...",WV_Parcels_All_Counties,,,,"MULTIPOLYGON (((-9018521.215 4771474.855, -901...",MULTIPOLYGON (((-9018521.215373708 4771474.854...,2702441
239740,1389768,,,,,,,,,,...,,,"{""type"": ""Feature"", ""properties"": {""SourceObje...",WV_Parcels_All_Counties,,,,"MULTIPOLYGON (((-9021851.620 4751798.350, -902...",MULTIPOLYGON (((-9021851.619823905 4751798.350...,1428838
221303,1371317,,,10,,,,0002,,,...,,109-9999.2,"{""type"": ""Feature"", ""properties"": {""SourceObje...",WV_Parcels_All_Counties,,,,"MULTIPOLYGON (((-8997137.507 4574790.984, -899...",MULTIPOLYGON (((-8997137.507201571 4574790.984...,1286020
239771,1389799,,,,,,,,,,...,,,"{""type"": ""Feature"", ""properties"": {""SourceObje...",WV_Parcels_All_Counties,,,,"MULTIPOLYGON (((-9024580.657 4746141.024, -902...",MULTIPOLYGON (((-9024580.656525046 4746141.024...,1101026
214270,1364284,,,10,,,,0002,,,...,,13-9999.2,"{""type"": ""Feature"", ""properties"": {""SourceObje...",WV_Parcels_All_Counties,,,,"MULTIPOLYGON (((-9020380.573 4605992.242, -902...",MULTIPOLYGON (((-9020380.57303671 4605992.2424...,973945
165381,1315395,,,08,,,,0000,13653.35 AC WTS BUFFALO SUR LESS ROW,ALLEGHENY RURAL NMTC LLC,...,,24-23,"{""type"": ""Feature"", ""properties"": {""SourceObje...",WV_Parcels_All_Counties,,,,"MULTIPOLYGON (((-9010031.987 4649352.471, -901...",MULTIPOLYGON (((-9010031.987067925 4649352.471...,965867
245660,1396114,,,,,,,,,,...,,,"{""type"": ""Feature"", ""properties"": {""SourceObje...",WV_Parcels_All_Counties,,,,"MULTIPOLYGON (((-9003410.571 4581059.147, -900...",MULTIPOLYGON (((-9003410.570567878 4581059.147...,908455
239822,1389850,,,,,,,,,,...,,,"{""type"": ""Feature"", ""properties"": {""SourceObje...",WV_Parcels_All_Counties,,,,"MULTIPOLYGON (((-9016749.996 4753064.133, -901...",MULTIPOLYGON (((-9016749.996499157 4753064.133...,842059
239755,1389783,,,,,,,,,,...,,,"{""type"": ""Feature"", ""properties"": {""SourceObje...",WV_Parcels_All_Counties,,,,"MULTIPOLYGON (((-9047386.556 4746248.706, -904...",MULTIPOLYGON (((-9047386.555941409 4746248.705...,817663


In [37]:
large_geojson = merged[merged['length'] >= 200000]
# large_geojson

In [38]:
# big_geoms = gpd.read_file('bad value.geojson')
# big_geoms = big_geoms.append(large_geojson)
# big_geoms.to_file('bad value.geojson')
large_geojson.shape[0]

51

In [136]:
# large_geojson

In [39]:
# ids = ['862319','883900']
# large_geojson = merged[merged['SourceObjectID'].isin(ids)]
# large_geojson
merged.drop(large_geojson.index, inplace=True)
merged.shape[0]

246234

In [40]:
merged = merged.drop(columns='length')
merged.columns

Index(['SourceObjectID', 'SourceGISID', 'SourceFullParcelID', 'CountyID',
       'DistrictID', 'TaxMapID', 'ParcelID', 'SuffixID',
       'FullLegalDescription', 'SourceFullOwnerName', 'SourceFullOwnerAddress',
       'SourceFullPhysicalAddress', 'SourceLabelName', 'GeoJSON', 'DataSource',
       'SourceDateTime', 'SysStartTime', 'SysEndTime', 'geometry', 'geom'],
      dtype='object')

In [41]:
# provide the needed credentials to access the database
server = 'm1data.database.windows.net' 
database = 'm1core.v3' 
username = 'Sayyed_dev' 
password = 'L32IsFgvF20grxRrXZ08'

In [42]:
# start building the connection to access the database and load the data to the dedicated table
cnxn = pyodbc.connect('DRIVER={SQL Server};SERVER='+server+';DATABASE='+database+';UID='+username+';PWD='+ password)
cursor = cnxn.cursor()

In [43]:
# for index, row in tqdm(merged.iterrows(), total=len(merged)):
#     cursor.execute("INSERT INTO dbo.WVTaxParcels_Import (SourceObjectID,SourceGISID,SourceFullParcelID,CountyID,DistrictID,TaxMapID,ParcelID,SuffixID,FullLegalDescription,SourceFullOwnerName,SourceFullOwnerAddress,SourceFullPhysicalAddress,SourceLabelName,GeoJSON,Geom,DataSource,SourceDateTime,SysStartTime,SysEndTime) VALUES (?,?,?,?,?,?,?,?,?,?,?,?,?,?,geometry::STGeomFromText(?, 3857),?,?,?,?)",
#                    row.SourceObjectID, row.SourceGISID, row.SourceFullParcelID, row.CountyID, row.DistrictID, row.TaxMapID, row.ParcelID, row.SuffixID,
#                    row.FullLegalDescription, row.SourceFullOwnerName, row.SourceFullOwnerAddress, row.SourceFullPhysicalAddress, row.SourceLabelName,
#                    row.GeoJSON, row.geom, row.DataSource, row.SourceDateTime, row.SysStartTime, row.SysEndTime)

# cnxn.commit()
# cursor.close()

In [44]:
# create a new GeoDataFrame to slice it and load data in chunks
gdf = merged.drop(columns='geometry')[0:25000]
# r = ['590027','590028','590029']
# gdf = merged[merged['SourceObjectID'].isin(r)]
# اخر رقم في السطر اللي فوق مش بيتحسب ... خليك فاكر
gdf.columns

Index(['SourceObjectID', 'SourceGISID', 'SourceFullParcelID', 'CountyID',
       'DistrictID', 'TaxMapID', 'ParcelID', 'SuffixID',
       'FullLegalDescription', 'SourceFullOwnerName', 'SourceFullOwnerAddress',
       'SourceFullPhysicalAddress', 'SourceLabelName', 'GeoJSON', 'DataSource',
       'SourceDateTime', 'SysStartTime', 'SysEndTime', 'geom'],
      dtype='object')

In [45]:
# assign the order of the columns to the new GeoDataFrame
gdf_order = ['SourceObjectID', 'SourceGISID', 'SourceFullParcelID', 'CountyID',
       'DistrictID', 'TaxMapID', 'ParcelID', 'SuffixID',
       'FullLegalDescription', 'SourceFullOwnerName', 'SourceFullOwnerAddress',
       'SourceFullPhysicalAddress', 'SourceLabelName', 'GeoJSON', 'geom', 'DataSource',
       'SourceDateTime', 'SysStartTime', 'SysEndTime']

gdf = gdf[gdf_order]
gdf.columns

Index(['SourceObjectID', 'SourceGISID', 'SourceFullParcelID', 'CountyID',
       'DistrictID', 'TaxMapID', 'ParcelID', 'SuffixID',
       'FullLegalDescription', 'SourceFullOwnerName', 'SourceFullOwnerAddress',
       'SourceFullPhysicalAddress', 'SourceLabelName', 'GeoJSON', 'geom',
       'DataSource', 'SourceDateTime', 'SysStartTime', 'SysEndTime'],
      dtype='object')

In [46]:
gdf.shape[0]

25000

In [47]:
# the SQL query that will load the data to the dedicated table on the database
sql_query = "INSERT INTO dbo.WVTaxParcels_Import (SourceObjectID,SourceGISID,SourceFullParcelID,CountyID,DistrictID,TaxMapID,ParcelID,SuffixID,FullLegalDescription,SourceFullOwnerName,SourceFullOwnerAddress,SourceFullPhysicalAddress,SourceLabelName,GeoJSON,Geom,DataSource,SourceDateTime,SysStartTime,SysEndTime) VALUES (?,?,?,?,?,?,?,?,?,?,?,?,?,?,geometry::STGeomFromText(?, 3857),?,?,?,?)"

In [48]:
# for index, row in tqdm(gdf.iterrows(), total=len(gdf)):
#     print(row['SourceObjectID'])
#     cursor.execute("INSERT INTO dbo.WVTaxParcels_Import (SourceObjectID,SourceGISID,SourceFullParcelID,CountyID,DistrictID,TaxMapID,ParcelID,SuffixID,FullLegalDescription,SourceFullOwnerName,SourceFullOwnerAddress,SourceFullPhysicalAddress,SourceLabelName,GeoJSON,Geom,DataSource,SourceDateTime,SysStartTime,SysEndTime) VALUES (?,?,?,?,?,?,?,?,?,?,?,?,?,?,geometry::STGeomFromText(?, 3857),?,?,?,?)",
#                    row.SourceObjectID, row.SourceGISID, row.SourceFullParcelID, row.CountyID, row.DistrictID, row.TaxMapID, row.ParcelID, row.SuffixID,
#                    row.FullLegalDescription, row.SourceFullOwnerName, row.SourceFullOwnerAddress, row.SourceFullPhysicalAddress, row.SourceLabelName,
#                    row.GeoJSON, row.geom, row.DataSource, row.SourceDateTime, row.SysStartTime, row.SysEndTime)

# cnxn.commit()
# cursor.close()

In [49]:
# define the chunk size as 1000 rows
data = [tuple(row) for row in gdf.itertuples(index=False)]
chunk_size = 1000
total_rows = len(data)

In [50]:
# iterate on the chunks and execute the SQL query to the chunks
with tqdm(total=total_rows, desc="Inserting rows") as pbar:
    for chunk in range(0, total_rows, chunk_size): 
        chunk_data = data[chunk:chunk+chunk_size]
        cursor.executemany(sql_query, chunk_data)
        pbar.update(len(chunk_data))

cnxn.commit()

cursor.close()

Inserting rows: 100%|██████████████████████████████████████████████████████████| 25000/25000 [4:37:25<00:00,  1.50it/s]


In [83]:
# batch_size = 1000
# total_rows = len(gdf)

# # Insert rows in batches
# for i in range(0, total_rows, batch_size):
#     print(i)
#     batch = gdf.iloc[i:i+batch_size]
#     values = [tuple(row) for row in batch.values]
#     cursor.executemany("INSERT INTO dbo.WVTaxParcels_Import (SourceObjectID,SourceGISID,SourceFullParcelID,CountyID,DistrictID,TaxMapID,ParcelID,SuffixID,FullLegalDescription,SourceFullOwnerName,SourceFullOwnerAddress,SourceFullPhysicalAddress,SourceLabelName,GeoJSON,Geom,DataSource,SourceDateTime,SysStartTime,SysEndTime) VALUES (?,?,?,?,?,?,?,?,?,?,?,?,?,?,geometry::STGeomFromText(?, 3857),?,?,?,?)", values)

# # Commit the transaction
# cnxn.commit()
# cursor.close()

In [84]:
# def load_chunk(chunk):
#     # Connect to SQL Server
#     cnxn = pyodbc.connect("DRIVER={SQL Server};SERVER=m1data.database.windows.net;DATABASE=m1core.v3;UID=Sayyed_dev;PWD=L32IsFgvF20grxRrXZ08")
#     cursor = cnxn.cursor()

#     # Your SQL query and table name
#     sql_query = sql_query
    
#     # Execute the query for the current chunk
#     cursor.executemany(sql_query, chunk)
#     cnxn.commit()

#     # Close the connection
#     cnxn.close()

In [85]:
# def load_data(data_chunk):
#     # Specify your chunk size
#     chunk_size = 20000

#     # Split the data into chunks
#     chunks = [data_chunk[i:i+chunk_size] for i in range(0, len(data_chunk), chunk_size)]

#     # Initialize tqdm for progress tracking
#     total_chunks = len(chunks)
#     with tqdm(total=total_chunks, desc="Inserting rows") as pbar:
#         for chunk in chunks:
#             # Load each chunk in parallel
#             load_chunk(chunk)
#             pbar.update(1)

In [86]:
# if __name__ == '__main__':
#     # Load your DataFrame here
#     data = gdf

#     # Split data into chunks for parallel processing
#     num_chunks = 5
#     chunk_size = len(data) // num_chunks
#     data_chunks = [data[i*chunk_size:(i+1)*chunk_size] for i in range(num_chunks)]
    
#     # Create a pool of processes
#     with Pool(cpu_count()) as pool:
#         pool.map(load_data, data_chunks)